In [1]:
import sys
%load_ext autoreload
%autoreload 2
sys.path.append("../base")
sys.path.append("../data_provider")
sys.path.append("../models")
from common import *
from data_provider import DataProvider
import simple_linear_regression as simple_lr
import lasso_linear_regression as lasso_lr
import ridge_linear_regression as ridge_lr
import catboost_regression as catboost_r

data_provider = DataProvider()
x_raw = data_provider.x_raw
y_raw = data_provider.y_raw